In [8]:
import matplotlib.pyplot as plt
from imutils import paths
import tensorflow as tf
import numpy as np
import cv2
import os

from bodypose.training.preprocessing import load_TFRecords_dataset  
from bodypose.training.architecture import MoveNet
import config as cfg

In [17]:
INPUT_SHAPE = (224, 224, 3)
STRIDES = (32, 16, 8, 4)
NUM_KPTS = cfg.N_KPTS

GRID_SIZE = INPUT_SHAPE[0] // STRIDES[-1]


In [6]:
from matplotlib.pyplot import grid


filePaths = list(paths.list_files("./dataset/tfrecords/coco/validation"))
np.random.shuffle(filePaths)
print(f"[INFO] Found {len(filePaths)} TFRecords.")

train_paths = filePaths[:-2]
valid_path = filePaths[-2:]

train_ds = load_TFRecords_dataset(
    train_paths, 
    batch_size = 32,
    target_size = INPUT_SHAPE[:2],
    grid_dim = GRID_SIZE,
    augmentations = [],
    roi_thresh = 0.0
    )

val_ds = load_TFRecords_dataset(
    train_paths, 
    batch_size = 32,
    target_size = INPUT_SHAPE[:2],
    grid_dim = GRID_SIZE,
    augmentations = [],
    roi_thresh = 0.0
    )

[INFO] Found 10 TFRecords.


In [11]:

model = MoveNet(
    input_shape = INPUT_SHAPE, 
    strides = STRIDES, 
    num_joints = NUM_KPTS, 
    alpha = .25, 
    use_depthwise = True
    )

img = (np.random.uniform(
    0, 255, (1,) + INPUT_SHAPE
    ).astype("uint8") / 255).astype(np.float32)

%timeit model(img)


(None, 56, 56, 17)
119 ms ± 2.64 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [12]:
model.summary()

Model: "move_net_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 backbone (Functional)       [(None, 7, 7, 1280),      248768    
                              (None, 14, 14, 144),               
                              (None, 28, 28, 48),                
                              (None, 56, 56, 48)]                
                                                                 
 FPN (Functional)            (None, 56, 56, 32)        49280     
                                                                 
 head (Functional)           [(None, 56, 56, 1),       16342     
                              (None, 56, 56, 34),                
                              (None, 56, 56, 17),                
                              (None, 56, 56, 34)]                
                                                                 
 post_processing (Functional  [(None, 17, 5),          0

In [13]:
224/4


56.0